In [49]:
#import libraries
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [50]:
#laod data
train_data=pd.read_csv("data/train.csv")
test_data=pd.read_csv("data/test.csv")
test_ids=test_data["PassengerId"]
train_data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [51]:
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [52]:
print(train_data.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [53]:
# cleaning the data
def clean_data(data):
    data=data.drop(["Ticket","PassengerId","Name", "Cabin"],axis=1)
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].mean(), inplace=True)
        
    data.Embarked.fillna("U", inplace=True)
    return data

In [54]:
#apply the above function
train_data=clean_data(train_data)
test_data=clean_data(test_data)
train_data.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


In [55]:
#label encoding
le = preprocessing.LabelEncoder()
columns = ["Sex", "Embarked"]

for col in columns:
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.transform(test_data[col])
    print(le.classes_)
      
train_data.head(3)

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2


In [56]:
# Splitting train and validation
y=train_data["Survived"]
X=train_data.drop(["Survived"],axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [57]:
model=LogisticRegression(random_state=0, max_iter=1000)
model.fit(X_train,y_train)

LogisticRegression(max_iter=1000, random_state=0)

In [58]:
#get predictions
val_predictions=model.predict(X_val)
val_accuracy=accuracy_score(y_val,val_predictions)
print(val_accuracy)

0.8100558659217877


In [59]:
# evaluationg the model on the test set
test_predictions=model.predict(test_data)

In [60]:
#save the predictions
df=pd.DataFrame({"PassengerId":test_ids.values,"Survived":test_predictions})
df.to_csv("Submission.csv",index=False)